1. Загрузить файл длиной не менее 2000 символов.

In [1]:
# open file, read file, get length, print length, return file content
def load_file(filepath: str):
    with open(filepath, 'r') as f:
        file_content = f.read()
        symbols_count = len(file_content)
        print(f'File contains {symbols_count} symbols')

    return file_content

In [2]:
filepath = 'text-file.txt'
file_text = load_file(filepath)


File contains 2166 symbols


2. Составить программу, которая считает число уникальных слов в тексте (без критерия
схожести)

In [3]:
# function to exclude non-akphabetic and non-spaces symbols 
from string import punctuation

# check that symbol is not punctuation sign
def is_target_symbol(symbol: str):
    return symbol not in punctuation

# exclude unnecessary symbols from text, split to words, make set of words, get set length
def calc_unique_words(text: str):
    words = ''.join(filter(is_target_symbol, text.lower())).split()
    uniques_count = len(set(words))
    print(f'There are {uniques_count} unique words in text')

    return uniques_count


In [4]:
uniques_count = calc_unique_words(file_text)


There are 214 unique words in text


3. Составить программу, которая считает число гласных и согласных букв.

In [5]:
# iterate through string and check if symbol in vowels string
# if symbol is non-vowel but alphabetic - it's consonant
def calculate_letters(string: str):
    vowels = 'aeiou'
    vowels_count, consonants_count = 0, 0

    for symbol in string:
        if symbol in vowels or symbol.lower() in vowels:
            vowels_count += 1
        elif symbol.isalpha():
            consonants_count += 1

    return vowels_count, consonants_count


In [6]:
vowels, consonants = calculate_letters(file_text)
print(f'Vowels count - {vowels}, consonats count - {consonants}')


Vowels count - 656, consonats count - 1054


4. Составить программу, которая считает число предложений, их длину и число (количество)
раз использования каждого слова в тексте (с критерием схожести, критерий схожести слов
выбрать самостоятельно, например, spacy (en_core_web_sm) или расстояние
Левенштейна).

5. Вывести 10 наиболее часто встречаемых слов. 
P.S. Рекомендую перед решением задания проанализировать задачу и обосновать
алгоритм ее решения в текстовом виде. В процессе написания кода использовать
комментарии.

Для подсчета предложений в тексте убираются точки из распространенных сокращений "Mrs.", "Mr." Заменяются другие знаки окончания предложения ("!", "?") на точку для упрощения разделения текста на предложения. Текст режется на предложения по разделителю ".", предложения делятся на слова. Для предложений, содержащих слова, выводится кол-во слов.

In [10]:
# calculate count of sentences and words in each one
def proc_sentences(string: str):
    # replace reduction to exclide point
    for reduction in ['Mrs.', 'Mr.']:
        proc_str = string.replace(reduction, reduction[:-1])

    # replace other sentences separators to simplify spliting by point
    for separator in ['!', '?']:
        proc_str = proc_str.replace(separator, '.')

    sentences_list = proc_str.split('.')
    sentences_count = 0

    for item in sentences_list:
        item = item.strip()

        if item:        
            sentences_count += 1
            sentence_length = len(item.split())
            print('Sentence:\n', item, '\nWords count -', sentence_length, '\n\n')

    print('Sentences count -', sentences_count)


In [8]:
file_text = load_file(filepath)
proc_sentences(file_text)


File contains 2166 symbols
Sentence:
 It details the story of a girl named Alice who falls through a rabbit hole into a fantasy world of anthropomorphic creatures 
Words count - 22 


Sentence:
 It is seen as an example of the literary nonsense genre 
Words count - 11 


Sentence:
 The artist John Tenniel provided 42 wood-engraved illustrations for the book 
Words count - 11 


Sentence:
 It received positive reviews upon release and is now one of the best-known works of Victorian literature; its narrative, structure, characters and imagery have had a widespread influence on popular culture and literature, especially in the fantasy genre 
Words count - 38 


Sentence:
 It is credited as helping end an era of didacticism in children's literature, inaugurating an era in which writing for children aimed to "delight or entertain" 
Words count - 26 


Sentence:
 The tale plays with logic, giving the story lasting popularity with adults as well as with children 
Words count - 17 


Sentence:

Для подсчета уникальных слов в тексте с критерием схожести сначала убираются знаки пунктуации. Текст (в нижнем регистре) разделяется на токены с помощью библиотеки spacy (легкая моделька en_core_web_sm). Составляется словарь: в качестве ключей - слова в первоначальной форме (с помощью spacy), значений - кол-ва вхождений слов-ключей в текст. Проверяется выделение символов '\n', '\t', '\s' в отдельный токен с последующем его игнорированием. 


Для выделения начальной формы слова можно было бы применять библиотеку nltk, в которой есть возможности определять корневую форму слова (stemming) или действительную форму слова (lemmatization), однако эти методы плохо работают с глаголами, предлогами и местоимениями. В библиотеке word_forms тоже есть возможность выделять начальную форму слова, но недостатками является отсутствие большого количества слов в ее словаре, а также низкое качество работы на предлогах. Методы сравнения слов по косинусному сходству и расстоянию Левенштейна являются менее обобщенными из-за необходимости подбирать и устанавливать пороги, чтобы считать слова схожими. Эти методы подошли бы больше для целенаправленного поиска максимально похожего слова для заданного. Библиотека spacy определяет начальные формы слов качественнее, поэтому используется для решения.

In [11]:
from string import punctuation, whitespace
import spacy

# find unique words count with spacy
def spacy_algorithm(text: str):
    # replace punctuation symbols in lowercase text, create tokens from text
    translator = str.maketrans('', '', punctuation)
    text_no_punct = text.lower().translate(translator)

    npl = spacy.load('en_core_web_sm')
    tokens = npl(text_no_punct)

    unique_text_words = dict()
    # fill dictionary with unique words, if word is't in word separators
    for token in tokens:
        if token.text not in whitespace:
            unique_text_words[token.lemma_] = unique_text_words[token.lemma_] + 1 if token.lemma_ in unique_text_words.keys() else 1

    unique_text_words = dict(sorted(unique_text_words.items(), key=lambda item: item[1], reverse=True))

    print(f'Unique words count is {len(unique_text_words)}')
    print('Words used in text:')
    for item in unique_text_words.items():
        print(item)

    return unique_text_words


In [12]:
file_text = load_file(filepath)
spacy_unique_words = spacy_algorithm(file_text)


File contains 2166 symbols
Unique words count is 201
Words used in text:
('the', 23)
('and', 10)
('in', 10)
('alice', 9)
('of', 8)
('a', 8)
('be', 7)
('as', 7)
('to', 6)
('carroll', 6)
('it', 5)
('have', 5)
('with', 5)
('she', 5)
('story', 4)
('for', 4)
('girl', 3)
('an', 3)
('book', 3)
('literature', 3)
('character', 3)
('which', 3)
('write', 3)
('child', 3)
('liddell', 3)
('age', 3)
('verse', 3)
('he', 3)
('name', 2)
('through', 2)
('into', 2)
('fantasy', 2)
('genre', 2)
('receive', 2)
('upon', 2)
('one', 2)
('its', 2)
('on', 2)
('end', 2)
('era', 2)
('well', 2)
('out', 2)
('game', 2)
('young', 2)
('adventure', 2)
('8', 2)
('at', 2)
('tell', 2)
('that', 2)
('his', 2)
('detail', 1)
('who', 1)
('fall', 1)
('rabbit', 1)
('hole', 1)
('world', 1)
('anthropomorphic', 1)
('creature', 1)
('see', 1)
('example', 1)
('literary', 1)
('nonsense', 1)
('artist', 1)
('john', 1)
('tenniel', 1)
('provide', 1)
('42', 1)
('woodengrave', 1)
('illustration', 1)
('positive', 1)
('review', 1)
('release', 1)

Как видно, среди часто встречаемых в тексте много слов, не несущих смысловой нагрузки, удаялем их из словаря до тех пор, пока не сформируется 10 наиболее распространенных в тексте слов со смыслом

In [13]:
# exclude stop words until 10 meaningful words not counted
def present_top(words_dict: dict):
    # may exclude all stop words form dictionary if it's needed, comment if with break
    TOP_COUNT = 10

    top_counter = 0
    keys_to_delete = list()

    print(TOP_COUNT, 'words frequently used in text:')
    # check word key in dictionary is in stop words, append stop words to list, than remove from dictionary
    for word_key in words_dict.keys():
        if word_key in spacy.lang.en.stop_words.STOP_WORDS:
            keys_to_delete.append(word_key)
            continue

        top_counter += 1
        # represent meaningful words right away to avoid iterate through dictionary twice
        print(word_key, words_dict[word_key])
        if top_counter == TOP_COUNT:
            break

    # exclude stop words
    for del_key in keys_to_delete:
        words_dict.pop(del_key)

    print(len(keys_to_delete), 'stop words were excluded')


In [14]:
present_top(spacy_unique_words)


10 words frequently used in text:
alice 9
carroll 6
story 4
girl 3
book 3
literature 3
character 3
write 3
child 3
liddell 3
15 stop words were excluded
